In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import sympy

In [ ]:
wavelength = 1
Ns = 9
dy = wavelength / 2
Lz = 5 * wavelength
Nr = 9
k = 2 * np.pi / wavelength

In [ ]:
def euclidean_distance(s, d) -> float:
    # s -> source point
    # d -> destination (receiving point)
    s = np.asarray(s)
    d = np.asarray(d)
    return np.linalg.norm(d - s)

def free_space_transfer_function(r_s, r_r, k) -> np.array:
    # r_s -> matrix of source points, shape (Ns, 2)
    # r_r -> matrix of recieving points, shape (Nr, 2)
    # k -> wavenumber
    # returns g -> matrix of floats, shape (Nr, Ns)
    g = np.zeros((len(r_r), len(r_s)), dtype=complex)
    for i, r in enumerate(r_r):
        for j, s in enumerate(r_s):
            d = euclidean_distance(s, r)
            # Check for eq 39
            if j == 2 and i == 0:
                check_d = d
            numerator = -np.exp(1j * k * d)
            denominator = 4 * np.pi * d
            g[i, j] = np.round(numerator / denominator, 5) # rounding here is questionable, but I need it for the checks
    return g, check_d

def sum_rule(g:np.array(complex)) -> float:
    S = np.asarray(0., dtype=np.float64)
    for i in g:
        for j in i:
            S += np.abs(j)**2
    return S

def matprint(mat, fmt="g"):
    col_maxes = [max([len(("{:"+fmt+"}").format(x)) for x in col]) for col in mat.T]
    for x in mat:
        for i, y in enumerate(x):
            print(("{:"+str(col_maxes[i])+fmt+"}").format(y), end="  ")
        print("")

In [ ]:
s = np.asarray([(0, i * (wavelength / 2)) for i in range(Ns)])
d = np.asarray([(Lz, i * (wavelength / 2)) for i in range(Nr)])
Gsr, check_d = free_space_transfer_function(s, d, k)
gsr = np.round(-4 * np.pi * Lz, 2)

In [ ]:
S = sum_rule(Gsr)

In [ ]:
# eq 51 check
print(np.allclose(S, 72.65 / (gsr**2), atol=1e-6))

In [ ]:
gsr_Gsrd_Gsr = np.round(gsr**2 * np.matmul(np.matrix.getH(Gsr), Gsr), 2)
matprint(gsr_Gsrd_Gsr)
print(gsr_Gsrd_Gsr.shape)

In [ ]:

from scipy.linalg import eigh

Gsr_Gsrd = np.matmul(np.matrix.getH(Gsr), Gsr)
eig_vals, eig_vect = np.linalg.eigh(Gsr_Gsrd)
eig_vect = np.round(np.flip(eig_vect, axis=1), 2)
eig_vals = np.round(np.flip(eig_vals), 5)
print(eig_vals.shape)
print(eig_vals)
print(eig_vals * (gsr**2))
print(np.sum(eig_vals * (gsr**2)))

In [ ]:
eig_vect.shape

In [ ]:
# Choose the central phase to be zero, normalize by it
def normalize_phase(eigenvector):
    # Choose middle index (zero-based)
    mid_idx = len(eigenvector) // 2
    # Extract phase of the middle entry
    phase = np.angle(eigenvector[mid_idx])
    # Normalize eigenvector by this phase
    return eigenvector * np.exp(-1j * phase)


# Normalize each eigenvector's phase
normalized_eigenvectors = np.column_stack([
    normalize_phase(eig_vect[:, j])
    for j in range(eig_vect.shape[1])
])

In [ ]:
x = np.linspace(0.2, Lz+0.2, 200)
y = np.linspace(-dy, dy * Ns, 200)
xx, yy = np.meshgrid(x,y)

points = np.stack([xx.ravel(), yy.ravel()], axis=-1)

In [ ]:
distance = []
for p in points:
    distance.append(euclidean_distance(s[3], p))
distance = np.stack(distance)
print(distance.shape)
distance = np.reshape(distance, xx.shape)
print(distance.shape)


In [ ]:
values = []

for m in range(0, 9):
    temp = []
    for p in points:
        h = normalized_eigenvectors[:, m]
        val = 0
        for i, source in enumerate(s):
            distance = euclidean_distance(source, p)
            numerator = np.exp(1j * k * distance) * h[i]
            val += numerator / distance
        val *= (-1 / (4 * np.pi))
        val *= np.sqrt(p[0])
        temp.append(val)
    values.append(temp)
values = np.asarray(values)

print(values.shape)
values = np.asarray([np.reshape(i, xx.shape) for i in values])
print(values.shape)

In [ ]:
for m in values:
    print(m.shape)

In [ ]:
for i,m in enumerate(values):
    fig, ax = plt.subplots(2,2, figsize=(10,10))
    
    
    ax[0][0].pcolormesh(xx, yy, m.real, cmap='jet')
    ax[0][0].scatter(s[:, 0], s[:, 1], color='black')
    ax[0][0].scatter(d[:, 0], d[:, 1], color='black')
    ax[0][0].set_title("Real")
    ax[0][0].set_xlabel(r"Z [$\lambda$]")
    ax[0][0].set_ylabel(r"Y [$\lambda$]")

    
    ax[0][1].pcolormesh(xx, yy,m.imag, cmap='jet')
    ax[0][1].scatter(s[:, 0], s[:, 1], color='black')
    ax[0][1].scatter(d[:, 0], d[:, 1], color='black')
    ax[0][1].set_title("Imaginary")
    ax[0][1].set_xlabel(r"Z [$\lambda$]")
    ax[0][1].set_ylabel(r"Y [$\lambda$]")
    
    ax[1][0].pcolormesh(xx, yy, np.abs(m), cmap='jet')
    ax[1][0].scatter(s[:, 0], s[:, 1], color='black')
    ax[1][0].scatter(d[:, 0], d[:, 1], color='black')
    ax[1][0].set_title("Magnitude")
    ax[1][0].set_xlabel(r"Z [$\lambda$]")
    ax[1][0].set_ylabel(r"Y [$\lambda$]")
    
    ax[1][1].pcolormesh(xx, yy, np.angle(m), cmap='hsv')
    ax[1][1].scatter(s[:, 0], s[:, 1], color='black')
    ax[1][1].scatter(d[:, 0], d[:, 1], color='black')
    ax[1][1].set_title("Phase")
    ax[1][1].set_xlabel(r"Z [$\lambda$]")
    ax[1][1].set_ylabel(r"Y [$\lambda$]")
    
    fig.suptitle("Mode {}".format(i))
    plt.tight_layout()
    fig.savefig("SS5_1_modes/mode_{:03d}.png".format(i), dpi=300)

## Let's get the source complex amplitudes

In [ ]:
plt.scatter(range(0,9), np.angle(normalized_eigenvectors[:,1]))

In [ ]:
np.angle(eig_vect[:,1][4])